# IEEE39 Powerflow

## Run simulation

### Import Libraries

In [ ]:
import subprocess, sys, os
import urllib.request

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt

#%matplotlib widget

### Get simulation data

In [ ]:
if not os.path.exists('ieee39-data'):
    os.mkdir('ieee39-data')

url = 'https://github.com/martinmoraga/dpsim_data/raw/main/IEEE39/matpower/ieee39.mat'
local_file_static = './ieee39-data/ieee39.mat'
urllib.request.urlretrieve(url, local_file_static)

### 1. Powerflow for initialization

In [ ]:
sim_name_pf = 'IEEE39_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader = matpower.Reader(mpc_file_path=local_file_static, mpc_name='ieee39')
mpc_reader.load_mpc(domain=matpower.Domain.PF)
system_pf = mpc_reader.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

### Read pf results

In [ ]:
dpsim_result_file = 'logs/' + sim_name_pf + '/' + sim_name_pf + '.csv'
ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

#### DPsim results

In [ ]:
dpsim_results = pd.DataFrame(columns=['Bus', 'Vm [pu]', 'Va [°]', 'P [MW]', 'Q [MVAr]'])
base_power = 1 #mw
for i in range(len(system_pf.nodes)):
    node_name = system_pf.nodes[i].name() #ex. N5
    node_number = node_name.replace('N', '')
    node_baseV = mpc_reader.mpc_bus_data.loc[mpc_reader.mpc_bus_data['bus_i'] == int(node_number), 'baseKV'].iloc[0] * 1e3
    w_mw = 1e-6
    dpsim_results.loc[i] = ([node_name] + [round(np.absolute(ts_dpsim_pf[node_name + '.V'].values[-1]) / node_baseV, 4)]
        + [round(np.angle(ts_dpsim_pf[node_name + '.V'].values[-1])*180/np.pi, 4)] 
        + [round(w_mw * np.real(ts_dpsim_pf[node_name + '.S'].values[-1]) / base_power, 4)] 
        + [round(w_mw * np.imag(ts_dpsim_pf[node_name + '.S'].values[-1]) / base_power, 4)])

dpsim_results

### MATPOWER Results

In [ ]:
matpower_pf_results = mpc_reader.get_pf_results()
matpower_pf_results

### Difference

In [ ]:
difference_vmag = matpower_pf_results['Vm [pu]'] - dpsim_results['Vm [pu]']
difference_vangle = matpower_pf_results['Va [°]'] - dpsim_results['Va [°]']

#base power = 100MW
difference_p = (matpower_pf_results['P [MW]'] - dpsim_results['P [MW]'])
difference_q = (matpower_pf_results['Q [MVAr]'] - dpsim_results['Q [MVAr]'])
difference = {"Bus": matpower_pf_results['Bus'], "V_mag [pu]": difference_vmag, "Va [°]": difference_vangle, "P [MW]": difference_p, "Q [MVAr]": difference_q}
difference = pd.DataFrame(difference)

difference

### Assertion

In [ ]:
tolerance = 1e-3

print(np.max(np.absolute(difference_p)))
assert(np.all(np.absolute(difference_p)<tolerance))
print(np.max(np.absolute(difference_q)))
assert(np.all(np.absolute(difference_q)<tolerance))

print(np.max(np.absolute(difference_vmag)))
assert(np.all(np.absolute(difference_vmag)<tolerance))

print(np.max(np.absolute(difference_vangle)))
assert(np.all(np.absolute(difference_vangle)<tolerance))

### Assert

In [ ]:
tolerance = 1e-3

print(np.max(np.absolute(difference_p)))
assert(np.all(np.absolute(difference_p)<tolerance))

print(np.max(np.absolute(difference_q)))
assert(np.all(np.absolute(difference_q)<tolerance))

print(np.max(np.absolute(difference_vmag)))
assert(np.all(np.absolute(difference_vmag)<tolerance))

print(np.max(np.absolute(difference_vangle)))
assert(np.all(np.absolute(difference_vangle)<tolerance))